In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics.pairwise import euclidean_distances
import math as mt
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from scipy.optimize import linear_sum_assignment

In [ ]:
SMALL_VALUE = 1e-18

In [ ]:
def adequacyCriterion(K,p,m,s,Lambda,D,U,G):
  result = []  
  for k in range(K):
    somaU = np.power(U[:,[k]],m).sum()   
    lam = 0
    auxL = []
    for j in range(p):    
      dist = []  
      lam = (np.power(Lambda[k][j],s))
      dj = np.matrix(D[j])
      for q in range(len(G[0])):
        e_ = G[k][q]
        dist.append(dj[:,[e_]].sum())
      auxL.append(lam*sum(dist))
    soma = sum(auxL)
    result.append(somaU * soma)
  return sum(result)

In [ ]:
def _members(K,n,p,Lambda,D,G,m=1.6):
	s = 1.0
	U = np.zeros((n, K), dtype=float)
	for i in range(n):
		for k in range(K):
			soma_tmp = []
			for h in range(K):
				dist_gk = []
				dist_gh = []
				for j in range(p):
					for q in range(len(G[0])):
						dist_gk.append(D[j][i][G[k][q]])
						dist_gh.append(D[j][i][G[h][q]])
				parte1 = sum((Lambda[k,:])**s) * sum(dist_gk)
				parte2 = sum((Lambda[h,:])**s) * sum(dist_gh)
				if parte1 == 0.0:
					parte1 = SMALL_VALUE
				if parte2 == 0.0:
					parte2 = SMALL_VALUE	
				tmp = (parte1/parte2) ** (2/(m-1))
				soma_tmp.append(tmp)	
			result = (sum(soma_tmp))**(-1)
			U[i][k] = result
	return U

In [ ]:
def prototypes(n, K, Lambda, m, s, q, D, U, p):
    l = []
    lp = []
    for i in range(n):
        lk = []
        for k in range(K):
            sumLD = 0
            sumU = (U[i][k])**m
            for P in range(p):
                mult = (Lambda[k][P]**s)*sum(D[P][i])
                sumLD+=mult
            lk.append(sumU*sumLD)
        lp.append(lk)
        aux = np.array(lp)
        l = aux.argsort(axis=0)

    aux = []
    g = []
    for Q in range(q):
        for x in range(K):
            if (l[Q][x] not in aux and l[Q][x] != -1):
                aux.append(l[Q][x])
                if(l[Q][x] == -1):
                    aux.append(l[Q+1][x])
                l[Q][x] = -1
            else:
                control = 0
                while (control == 0):
                    Q = Q + 1
                    if (l[Q][x] not in aux and l[Q][x] != -1):
                        aux.append(l[Q][x])
                        if (l[Q][x] == -1):
                            aux.append(l[Q + 1][x])
                        l[Q][x] = -1
                        control = 1
            Q = 0
        g.append(aux)
        aux=[]
    G = np.transpose(g)
    return G

In [ ]:
def lambda_K(n, K, m, q, D, U, p, G):
  lambda_kj = []
  l = []
  part1 = 1
  gk = 0
  for k in range(K):
    sumU = 0
    #Somatório de todos os exemplos Ui de cada K
    for i in range(n):
      sumU += U[i][k] ** m
    #Somatório de todos os exemplos Ei da matriz de Dj em relação ao exemplo do grupo Gk  
    vetD = []
    sumD = []
    prod = []
    
    for d in range (p):
      #Pega o representante do grupo Gk
      gk = G[k]
      #Soma todos os exemplos Ei da matriz j em relação ao exemplo gk                                                  
      vetD.append(sum(D[d][gk][0:len(D[d])]))

    for v in vetD:
      sumD.append (sum(v))             
      
    #Função para calcular o produto de Ui com o somatório de cada matriz
    for value in sumD:
      prod.append(sumU * value)
    #Variável que calcula o produtório entre todos os produtos
    part1 = np.product(prod)**(1/p)

    #Função que realiza a divisão entre o produtório e os produtos individuais de cada matriz
    res = []
    count = 0
    for pdt in prod:
      res.append(part1/pdt)
      count +=1
    l.append(res)
  lambda_kj = np.reshape(l, (K, p))
  return lambda_kj

In [ ]:
def PartEntropy(U, n, K):
  PE = 0
  for k in range (K):
    for i in range (n):
      PE += (U[i][k]) * (mt.log(U[i][k])/n)
  return -PE

In [ ]:
def PartCoef(U, n, K):
  PC = 0
  for k in range (K):
    for i in range (n):
      PC += (U[i][k])**2 /n
  return PC

In [ ]:
def ModPartCoef(U, n, K):
  MPC = 1 - K/(K-1)*(1-PartCoef(U, n, K))
  return MPC

In [ ]:
def cluster_acc(y_true, y_pred):
    """
    Calculate clustering accuracy. Require from scipy.optimize import linear_sum_assignment
    # Arguments
        y: true labels, numpy.array with shape `(n_samples,)`
        y_pred: predicted labels, numpy.array with shape `(n_samples,)`
    # Return
        accuracy, in [0,1]
    """
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    ind = linear_sum_assignment(w.max() - w)
    ind = np.asarray(ind)
    ind = np.transpose(ind)
    return sum([w[i, j] for i, j in ind]) * 1.0 / y_pred.size

In [ ]:
def MFCMddRWL(D,K=10,q=1, max_t=150, m=1.6, error=(10**(-10))):
	"""
	O algoritmo recebe os seguintes parâmetros:
	D - matriz de dissimilaridade 
	K - número de clusters (default= 10)
	q - cardinalidade de G_k (default=1)
	t - total de iterações (default=150)
	m - ...					(default=1.6)
	error - que é o error  (default =1e-10) 
	"""

	"""
	INICIALIZAÇÃO 
	"""
	#cardinalidade q de G_k
	
	t = 0
	s = 1.0

	#quantidade de matrizes
	p = len(D)
	n = len(D[0])

	#MFCMddRWL-P - Matching function 7 -  o produto dos pesos é igual a 1
	#matriz de pesos relevantes Lambda_kp
	Lambda = np.array([[1 for pp in range(p)] for k in range(K)])

	#vetor de protótipos 
	#G = np.random.choice(n, size=K,replace=False)	
	G = np.random.choice(n,[K,q],replace=False)

	#inicialização da matriz U
	U = _members(K,n,p,Lambda,D,G,m)
 
	# calcule o critério de adequação J⁽⁰⁾
	J = adequacyCriterion(K,p,m,s,Lambda,D,U,G)
	Jold = 0

	count = 0

	while((abs(J - Jold) >= error) and (t < max_t)):
		#(2) Step 1: Calculo dos melhores protótipos

		t = t + 1

		# U e Lambda são fixados
		#Calculando Gk = G* e E^(q)
		G = prototypes(n, K, Lambda, m, s, q, D, U, p)
		
		# (3) Step 2: computando a melhor matriz de pesos relevantes 
		#matriz de pesos relevantes Lambda_kp
		#U^(t-1) e G são fixados 
		Lambda = lambda_K(n, K, m, q, D, U, p, G)
						 
		# (4) Step 3: definição da melhor partição fuzzy
		# o vetor G e Lambda são fixos 
		U = _members(K,n,p,Lambda,D,G,m)
		 
		# critério de parada 
		# calcule o critério de adequação J^(t) e Jold = J(t - 1)
		Jold = J
		J = adequacyCriterion(K,p,m,s,Lambda,D,U,G)
	return G, U, J

In [ ]:
def main():
	#dataset
	view1 = pd.read_csv('https://raw.githubusercontent.com/Francimaria/Machine-Learning-Python/master/mfeat-fac.csv', header=None, delim_whitespace=True)
	view2 = pd.read_csv('https://raw.githubusercontent.com/Francimaria/Machine-Learning-Python/master/mfeat-fou.csv', header=None, delim_whitespace=True)
	view3 = pd.read_csv('https://raw.githubusercontent.com/Francimaria/Machine-Learning-Python/master/mfeat-kar.csv', header=None, delim_whitespace=True)

	#normalização
	# max 
	view1 = preprocessing.normalize(view1,'max')
	view2 = preprocessing.normalize(view2,'max')
	view3 = preprocessing.normalize(view3,'max')
 

	#Euclidian distance 
	# Cada matriz tem o tamanho nxn (2000x2000)
	D1 = euclidean_distances(view1,view1)
	D2 = euclidean_distances(view2, view2)
	D3 = euclidean_distances(view3,view3)
 
 
	#D é composto pelas 3 matrizes de dissimilaridade 
	#Assim D é um vetor de p matrizes 
	D = [D1,D2,D3]

	#função
	#rodar 100 vezes para obter uma partição fuzzy em 10 grupos
	prototypes = []
	members = []
	objective = []
	for i in range(100):
		print(i)
		G, U, J = MFCMddRWL(D,K=10,q=2)
		prototypes.append(G)
		members.append(U)
		objective.append(J)
	# index da função objetivo com menor valor
	index_array = np.argmin(objective, axis=-1)
	G = prototypes[index_array]
	U = members[index_array]
	J = objective[index_array]

	# Salve em csv as matrizes
	np.savetxt("/content/sample_data/G_tres_matrizes.csv",G.astype(int), fmt='%i',delimiter=",")
	np.savetxt("/content/sample_data/U_tres_matrizes.csv",U,delimiter=",")

	K = 10
	n = len(D[0])

  #partição CRISP
	cluster_membership = np.argmax(U, axis=1) 
	grupos  = [[i for i in range(len(cluster_membership)) if k == cluster_membership[i]] for k in range(K) ]
	#mostra a quantidade de elementos em cada grupo e o índice dos elementos de cada grupo
	print("** DESCRIÇÃO DOS GRUPOS **")
	print("SET-MEDOIDS (protótipo)")
	print(G)
	print("PARTIÇÃO CRISP E A QUANTIDADE DE OBJETOS")
	print("_________________________________________________________")
	for k in range(K):
		print("GRUPO", k+1,":",len(grupos[k]))
		print("Obj:", (grupos[k]))
		print("_________________________________________________________")
	
	# Salve em csv as labels da partição crisp
	np.savetxt("/content/sample_data/cluster_membership_tres_m.csv",cluster_membership.astype(int), fmt='%i', delimiter=",")
	
	#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html
	#índice de Rand corrigido adjusted_rand_score(labels_true, labels_pred)	
	#The first 200 patterns are of class `0', followed by sets of 200 patterns for each of the classes `1' - `9'.
	
	labels_= [i for i in range(10) for _ in range(200)]
	y_true = np.asarray(labels_, dtype=np.int64)

	rc = adjusted_rand_score(labels_, cluster_membership)
	print("Índice de rand corrigido: ", rc)
 
  #F-measure e erro de classificação (atribuição).	
	f1_measure_macro = f1_score(labels_, cluster_membership, average='macro')
	print("F-measure Macro: ",f1_measure_macro)
	
	f1_measure_micro = f1_score(labels_, cluster_membership, average='micro')
	print("F-measure Micro: ", f1_measure_micro )

	f1_measure_ = f1_score(labels_, cluster_membership, average=None)
	print("F-measure: ",f1_measure_)

	#acc = accuracy_score(labels_, cluster_membership) 
	#print("Erro de classificação: ", (1-acc))
 
	c_acc = cluster_acc(y_true, cluster_membership)
	print("Erro de Classificação do Cluster", (1-c_acc))
 
 	#Coeficientes
	PE = PartEntropy(U, n, K)
	print("Partition Entropy:", PE)
	MPC = ModPartCoef(U, n, K)
	print("Modified Partition Coefficient:", MPC)

In [ ]:
main()